In [3]:
import pandas as pd
import requests 
import json

In [4]:
#coordinates for the 22 WAQI Stations
coordinates = [[19.044, 73.0325],
 [19.1135051, 73.008978],
 [19.053536, 72.84643],
 [19.1375, 72.915056],
 [19.23241, 72.86895],
 [19.1653323, 72.922099],
 [18.96702, 72.84214],
 [18.9936162, 72.8128113],
 [19.10861, 72.83622],
 [19.192056, 72.9585188],
 [19.25292, 73.142019],
 [19.047, 72.8746],
 [19.2058, 72.8682],
 [19.175, 72.9419],
 [19.19709, 72.82204],
 [19.008751, 73.01662],
 [19.000083, 72.813993],
 [19.072830200195, 72.882606506348],
 [19.0863, 72.8888],
 [19.10078, 72.87462],
 [19.2243333, 72.8658113],
 [19.04946, 72.923]]

In [5]:
weather_lat = []
weather_lon = []

for x in range(0, len(coordinates)):
  try:
    # the coordinates from the aq measuring station
    lat = str(coordinates[x][0])
    lon = str(coordinates[x][1])

    #create url for each station
    api_url = "https://api.open-meteo.com/v1/forecast?latitude=" + lat + "&longitude=" + lon + \
    "&hourly=temperature_2m,relativehumidity_2m,rain,weathercode,surface_pressure," + \
    "cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,windgusts_10m&forecast_days=1"

    #accessing the api and pulling information
    headers = {"accept": "application/json"}

    response = requests.get(api_url, headers = headers)

    #confirm api pull request is sucessful
    print(f"Number {x}: status_code: {response.status_code}")

    #"open-meteo.com" closest weather coordinate to aq measuring station coordinate
    temp_lat = response.json()["latitude"]
    temp_lon = response.json()["longitude"]

    #add to list
    weather_lat.append(temp_lat)
    weather_lon.append(temp_lon)

  except:
    print(f'The Station with coordinates, {lat}, {lon} did not have data')

Number 0: status_code: 200
Number 1: status_code: 200
Number 2: status_code: 200
Number 3: status_code: 200
Number 4: status_code: 200
Number 5: status_code: 200
Number 6: status_code: 200
Number 7: status_code: 200
Number 8: status_code: 200
Number 9: status_code: 200
Number 10: status_code: 200
Number 11: status_code: 200
Number 12: status_code: 200
Number 13: status_code: 200
Number 14: status_code: 200
Number 15: status_code: 200
Number 16: status_code: 200
Number 17: status_code: 200
Number 18: status_code: 200
Number 19: status_code: 200
Number 20: status_code: 200
Number 21: status_code: 200


In [6]:
lat_list = []
lon_list = []

for n in range(0, len(coordinates)):
  aq_lat = coordinates[n][0]
  aq_lon = coordinates[n][1]

  lat_list.append(aq_lat)
  lon_list.append(aq_lon)

In [7]:
data = {"aq_latitude": lat_list ,
        "aq_longitude": lon_list ,
        "weather_latitude": weather_lat,
        "weather_longitude": weather_lon
        }

df = pd.DataFrame(data)
df.head()

,aq_latitude,aq_longitude,weather_latitude,weather_longitude
0,19.044000,73.032500,19.125,73.000
1,19.113505,73.008978,19.125,73.000
2,19.053536,72.846430,19.125,72.875
3,19.137500,72.915056,19.125,72.875
4,19.232410,72.868950,19.250,72.875


Checking For Duplicate Weather Coordinate
In the code below I checked how many weather coordinates are unique by using the pandas duplicated function. Essentially the code populates a True if the weather coordinate pair("weather_lat and weather_lon) has appeared before in a different row.

Pandas duplicated function documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html

In [8]:
df["is_duplicate"] = df[["weather_latitude", "weather_longitude"]].duplicated()
df.head()

,aq_latitude,aq_longitude,weather_latitude,weather_longitude,is_duplicate
0,19.044000,73.032500,19.125,73.000,False
1,19.113505,73.008978,19.125,73.000,True
2,19.053536,72.846430,19.125,72.875,False
3,19.137500,72.915056,19.125,72.875,True
4,19.232410,72.868950,19.250,72.875,False


In [16]:
unique_latitude = []
unique_longitude = []
unique_pairing = []
pairing_label = 0

for j in range(0, len(df)):
  if df["is_duplicate"][j] == False:
    print(f'row {j}: Has unique Weather coordinates')
    unique_lat = df["weather_latitude"][j]
    unique_lon = df["weather_longitude"][j]
    pairing_label += 1
    
    unique_pairing.append(pairing_label)
    unique_latitude.append(unique_lat)
    unique_longitude.append(unique_lon)
  
  else:
    unique_pairing.append(0)

print('----------------------------------------------------------')
print(f'{unique_latitude}')
print(f'{unique_longitude}')
print(f'{unique_pairing}')

row 0: Has unique Weather coordinates
row 2: Has unique Weather coordinates
row 4: Has unique Weather coordinates
row 6: Has unique Weather coordinates
row 9: Has unique Weather coordinates
row 10: Has unique Weather coordinates
----------------------------------------------------------
[19.125, 19.125, 19.25, 18.875, 19.25, 19.25]
[73.0, 72.875, 72.875, 73.0, 73.0, 73.125]
[1, 0, 2, 0, 3, 0, 4, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [18]:
df["pairing"] = unique_pairing
df.head()

,aq_latitude,aq_longitude,weather_latitude,weather_longitude,is_duplicate,pairing
0,19.044000,73.032500,19.125,73.000,False,1
1,19.113505,73.008978,19.125,73.000,True,0
2,19.053536,72.846430,19.125,72.875,False,2
3,19.137500,72.915056,19.125,72.875,True,0
4,19.232410,72.868950,19.250,72.875,False,3


In [38]:
temp_list = df["pairing"].to_list()

for k in range(0, len(df)):  
  if df["is_duplicate"][k] == True:
    for n in range(0, len(df)):
      if df["weather_latitude"][k] == df["weather_latitude"][n] and df["weather_longitude"][k] == df["weather_longitude"][n]:
        temp_list[k] = df["pairing"][n]
        break

df["pairing"] = temp_list
df

,aq_latitude,aq_longitude,weather_latitude,weather_longitude,is_duplicate,pairing
0,19.044000,73.032500,19.125,73.000,False,1
1,19.113505,73.008978,19.125,73.000,True,1
2,19.053536,72.846430,19.125,72.875,False,2
3,19.137500,72.915056,19.125,72.875,True,2
4,19.232410,72.868950,19.250,72.875,False,3
5,19.165332,72.922099,19.125,72.875,True,2
6,18.967020,72.842140,18.875,73.000,False,4
7,18.993616,72.812811,19.125,72.875,True,2
8,19.108610,72.836220,19.125,72.875,True,2
9,19.192056,72.958519,19.250,73.000,False,5


In [51]:
part_2 = df[["aq_latitude", "aq_longitude", "pairing"]]
part_2["type"] = "air_quality"
part_2 = part_2.rename(columns={"aq_latitude":"latitude", "aq_longitude":"longitude"})
part_2.head()

<ipython-input-51-8670d5cc79f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part_2["type"] = "air_quality"


,latitude,longitude,pairing,type
0,19.044000,73.032500,1,air_quality
1,19.113505,73.008978,1,air_quality
2,19.053536,72.846430,2,air_quality
3,19.137500,72.915056,2,air_quality
4,19.232410,72.868950,3,air_quality


In [48]:
part_1 = df.loc[(df["is_duplicate"] == False), ("weather_latitude", "weather_longitude", "pairing")]
part_1["type"] = "weather"
part_1 = part_1.rename(columns={"weather_latitude":"latitude", "weather_longitude":"longitude"})
part_1

,latitude,longitude,pairing,type
0,19.125,73.000,1,weather
2,19.125,72.875,2,weather
4,19.250,72.875,3,weather
6,18.875,73.000,4,weather
9,19.250,73.000,5,weather
10,19.250,73.125,6,weather


In [49]:
final_coordiates = part_1.merge(part_2, how="outer")
final_coordiates

,latitude,longitude,pairing,type
0,19.125000,73.000000,1,weather
1,19.125000,72.875000,2,weather
2,19.250000,72.875000,3,weather
3,18.875000,73.000000,4,weather
4,19.250000,73.000000,5,weather
5,19.250000,73.125000,6,weather
6,19.044000,73.032500,1,air_quality
7,19.113505,73.008978,1,air_quality
8,19.053536,72.846430,2,air_quality
9,19.137500,72.915056,2,air_quality


In [52]:
final_coordiates.to_csv("live_weather_stations.csv")